# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

In [3]:
#Your code here
df = pd.read_csv('homepage_actions.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [4]:
df.sort_values('id').head(10)

,timestamp,id,group,action
4390,2016-11-26 14:38:47.682931,182988,experiment,view
5639,2016-12-14 13:19:00.940677,182994,control,view
5640,2016-12-14 13:20:29.564080,182994,control,click
376,2016-09-29 23:04:42.722325,183089,control,view
6568,2016-12-27 10:13:58.808014,183136,experiment,view
3941,2016-11-20 08:00:22.434253,183141,experiment,click
3940,2016-11-20 07:58:52.768641,183141,experiment,view
4247,2016-11-24 02:31:00.749999,183248,control,click
4246,2016-11-24 02:30:11.939588,183248,control,view
3167,2016-11-09 09:55:28.020462,183283,experiment,view


In [5]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [6]:
set_view = set(df[df.action=='view']['id'])
set_click = set(df[df.action=='click']['id'])

print(f"""Count who viewed but not cliked: {len(set_view - set_click)}""")
print(f"""Count who clicked but not vieved: {len(set_click - set_view)}""")
print("Count Intersection:", len(set_click & set_view))

Count who viewed but not cliked: 4468
Count who clicked but not vieved: 0
Count Intersection: 1860


In [7]:
df['group'].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [8]:
set_exp = set(df[df.group=='experiment']['id'])
set_ctrl = set(df[df.group=='control']['id'])
print("Count Intersection:", len(set_exp & set_ctrl))

Count Intersection: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [9]:
#Your code here
df['count'] = 1

In [10]:
df.head(10)

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1
5,2016-09-24 20:32:25.712659,681598,experiment,view,1
6,2016-09-24 20:39:03.248853,522116,experiment,view,1
7,2016-09-24 20:57:20.336757,349125,experiment,view,1
8,2016-09-24 20:58:01.948663,349125,experiment,click,1
9,2016-09-24 21:00:12.278374,560027,control,view,1


In [11]:

control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)
control.head()

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [12]:
experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)
experiment.head()

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0


In [13]:
print("Control:")
control.agg(['sum', 'mean'])

Control:


,click,view
sum,932.000000,3332.0
mean,0.279712,1.0


In [14]:
print("Experiment:")
experiment.agg(['sum', 'mean'])

Experiment:


,click,view
sum,928.000000,2996.0
mean,0.309746,1.0


In [23]:
import flatiron_stats as fs
p = fs.p_value_welch_ttest(control.click, experiment.click)
print('p-value = {}'.format(p))

p-value = 0.004466402814337078


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [26]:
#Your code here
ctrl_click_rate = control.click.mean()
exp_expected_click=len(experiment)*ctrl_click_rate
print('# of expected clicks in experiment group = {}'.format(exp_expected_click))

# of expected clicks in experiment group = 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [31]:
#Your code here
n = len(experiment)
p = ctrl_click_rate
std = np.sqrt(n*p*(1-p))
print(std)

24.568547907005815


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [36]:
#Your code here
z = (experiment['click'].sum() - exp_expected_click)/std
p_val = 1 - stats.norm.cdf(z)

print('z-statics = {}'.format(z))
print('p-val = {}'.format(p_val))

z-statics = 3.6625360854823588
p-val = 0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Your analysis here

In [38]:
# p-val is similar in both methods
# Both methods reject the null-hypothesis, p-val<0.05

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.